In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [ ]:
def polyval(coefficients, x):
    total = 0
    for coeff in range(len(coefficients)):
        total = total + coefficients[-(coeff+1)]*(x**coeff)
    return np.float32(total)

def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*polyval(a_liq,np.maximum(c_liq,T-T0))

def eiceOriginal(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return np.transpose(np.where(T>c_ice[0],eliq(T),\
                   np.transpose(np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*polyval(a_ice,T-T0)))))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*polyval(a_ice,T-T0)))

def esatOriginal(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return np.transpose(np.where(T>T0,eliq(T),np.transpose(np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))))

def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def qv(T,RH,P0,PS,hyam,hybm):

    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)

    T = np.float32(T)
    RH = np.float32(RH)
    p = np.float32(p)

    return R*esat(T)*RH/(Rv*p)
    # DEBUG 1
    # return esat(T)

def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = np.float32(T)
    qv = np.float32(qv)
    p = np.float32(p)
    
    return Rv*p*qv/(R*esat(T))

In [ ]:
datasets = !ls
datasets = [x for x in datasets if "h1" in x]
spData = xr.open_mfdataset(datasets)

In [ ]:
nntbp = np.array(spData["NNTBP"])
nnqbp = np.array(spData["NNQBP"])
p0 = np.array(spData["P0"])
ps = np.array(spData["NNPS"])
hyam = np.array(spData["hyam"])
hybm = np.array(spData["hybm"])
relhum = np.array(spData["RELHUM"])
tphystnd = np.array(spData["TPHYSTND"])
phq = np.array(spData["PHQ"])

p0 = np.array(list(set(p0)))

newhum = np.zeros((spData["time"].shape[0],\
                     spData["lev"].shape[0], \
                     spData["lat"].shape[0], \
                     spData["lon"].shape[0]))
lats = np.array(spData["lat"])
lons = np.array(spData["lon"])

for i in range(len(lats)):
    for j in range(len(lons)):
        latIndex = i
        lonIndex = j
        R = np.float32(287.0)
        Rv = np.float32(461.0)
        p = p0 * hyam + ps[:, None, latIndex, lonIndex] * hybm # Total pressure (Pa)

        T = np.float32(nntbp[:, :, latIndex, lonIndex])
        qv = np.float32(nnqbp[:, :, latIndex, lonIndex])
        p = np.float32(p)
        newhum[:,:, latIndex, lonIndex] = Rv*p*qv/(R*esat(T))

In [ ]:
nntbp = np.moveaxis(nntbp[1:,:,:,:],0,1)
nnqbp = np.moveaxis(nnqbp[1:,:,:,:],0,1)
lhflx = np.array([np.array(spData["LHFLX"])])[:,:-1,:,:]
shflx = np.array([np.array(spData["SHFLX"])])[:,:-1,:,:]
ps = np.array([np.array(ps)])[:,1:,:,:]
solin = np.array([np.array(spData["SOLIN"])])[:,1:,:,:]
newhum = np.moveaxis(newhum[1:,:,:,:],0,1)
oldhum = np.moveaxis(relhum[1:,:,:,:],0,1)
tphystnd = np.moveaxis(tphystnd[1:,:,:,:],0,1)
phq = np.moveaxis(phq[1:,:,:,:],0,1)
print(nntbp.shape)
print(nnqbp.shape)
print(lhflx.shape)
print(shflx.shape)
print(ps.shape)
print(solin.shape)
print(newhum.shape)
print(oldhum.shape)
print(tphystnd.shape)
print(phq.shape)

In [ ]:
nnInput = np.concatenate((nntbp, \
                          nnqbp, \
                          lhflx, \
                          shflx, \
                          ps, \
                          solin, \
                          newhum, \
                          oldhum, \
                          tphystnd, \
                          phq))
nnInput.shape

In [ ]:
suffix = !pwd
suffix = re.sub("^[^_]+_","", suffix[0])

In [ ]:
fileName = 'nnInput_' + suffix + '.npy'
with open(fileName, 'wb') as f:
    np.save(f, nnInput)

In [ ]:
errors = (newhum-oldhum/100).flatten()
result = "Mean error: " + str(np.mean(errors)) + "\n"
result = result + "Variance: " + str(np.var(errors)) + "\n"
result = result + "nntbp.shape: " + str(nntbp.shape) + "\n"
result = result + "nnqbp.shape: " + str(nnqbp.shape) + "\n"
result = result + "lhflx.shape: " + str(lhflx.shape) + "\n"
result = result + "shflx.shape: " + str(shflx.shape) + "\n"
result = result + "ps.shape: " + str(ps.shape) + "\n"
result = result + "solin.shape: " + str(solin.shape) + "\n"
result = result + "newhum.shape: " + str(newhum.shape) + "\n"
result = result + "oldhum.shape: " + str(oldhum.shape) + "\n"
result = result + "tphystnd.shape: " + str(tphystnd.shape) + "\n"
result = result + "phq.shape: " + str(phq.shape) + "\n"
result = result + "nnInput.shape: " + str(nnInput.shape) + "\n"

In [ ]:
result

In [ ]:
diagnostics = 'diagnostics_' + suffix + '.txt'
with open(diagnostics, 'a') as fp:
    fp.write(result)